In [1]:
import spacy
nlp = spacy.load('en_core_web_md')

In [2]:
import neuralcoref
neuralcoref.add_to_pipe(nlp)

In [1]:
engParts = []
temp = ''
with open('../Data/Datasets/Europarl/Documents/ep-11-06-23-004/text/ep-11-06-23-004-en.txt') as f:
    for index, line in enumerate(f):
        temp = temp + line
        if (index + 1) % 18 == 0:
            engParts.append(temp)
            temp = ''

In [4]:
docs = []
for part in engParts:
    doc = nlp(part)
    docs.append(doc)


In [5]:
sweParts = []
sweCorp = ''
with open('../Data/Datasets/Europarl/Documents/ep-11-06-23-004/text/ep-11-06-23-004-sv.txt') as f:
    for index, line in enumerate(f):
        sweCorp = sweCorp + line
        if (index + 1) % 18 == 0:
            sweParts.append(sweCorp)
            sweCorp = ''


In [6]:
aliTextParts = []
temp = ''
with open('../Data/Datasets/Europarl/Documents/ep-11-06-23-004/alignments/ali.sym') as f:
    for index, line in enumerate(f):
        temp = temp + line
        if (index + 1) % 18 == 0:
            aliTextParts.append(temp)
            temp = ''


In [7]:
tokenizerEng = spacy.load('en_core_web_md', disable=['tagger', 'parser', 'ner'])
tokenizerSwe = spacy.load('../Models/SwedishModel', disable=['tagger', 'parser', 'ner'])

In [8]:
alignmentDicts = []
for engPart, swePart, aliPart in zip(engParts, sweParts, aliTextParts):
    aliDict = {}
    startTokenEng = 0
    startTokenSwe = 0
    engLines = engPart.split('\n')
    for index, line in enumerate(engLines):
        engLines[index] = line + '\n'
    engLines.pop()

    sweLines = swePart.split('\n')
    for index, line in enumerate(sweLines):
        sweLines[index] = line + '\n'
    sweLines.pop()

    aliLines = aliPart.split('\n')
    for index, line in enumerate(aliLines):
        aliLines[index] = line + '\n'
    aliLines.pop()
    for engLine, sweLine, aliLine in zip(engLines, sweLines, aliLines):
        tokEng = tokenizerEng(engLine)
        tokSwe = tokenizerSwe(sweLine)

        alignmentListTemp = aliLine.split()
        for ali in alignmentListTemp:
            aliPair = ali.split('-')
            key = str(int(aliPair[0]) + startTokenEng)
            if aliDict.get(key) is None:
                aliDict[key] = [int(aliPair[1]) + startTokenSwe]
            else:
                aliDict.get(key).append(int(aliPair[1]) + startTokenSwe)

        startTokenEng = startTokenEng + len(tokEng)
        startTokenSwe = startTokenSwe + len(tokSwe)
    alignmentDicts.append(aliDict)


In [9]:
"""startTokenEng = 0
startTokenSwe = 0
alignments = {}

with open('../Data/Datasets/Europarl/Documents/ep-11-06-23-004/text/ep-11-06-23-004-en.txt') as engDoc:
    with open('../Data/Datasets/Europarl/Documents/ep-11-06-23-004/alignments/ali.sym') as fa:
        with open('../Data/Datasets/Europarl/Documents/ep-11-06-23-004/text/ep-11-06-23-004-sv.txt') as sweDoc:
            for lineAli in fa:
                lineEngDoc = engDoc.readline()
                lineSweDoc = sweDoc.readline()
                tokEng = tokenizerEng(lineEngDoc)
                tokSwe = tokenizerSwe(lineSweDoc)

                alignmentListTemp = lineAli.split()
                for ali in alignmentListTemp:
                    aliPair = ali.split('-')
                    key = str(int(aliPair[0]) + startTokenEng)
                    if alignments.get(key) is None:
                        alignments[key] = [int(aliPair[1]) + startTokenSwe]
                    else:
                        alignments.get(key).append(int(aliPair[1]) + startTokenSwe)

                startTokenEng = startTokenEng + len(tokEng)
                startTokenSwe = startTokenSwe + len(tokSwe)"""


"startTokenEng = 0\nstartTokenSwe = 0\nalignments = {}\n\nwith open('../Data/Datasets/Europarl/Documents/ep-11-06-23-004/text/ep-11-06-23-004-en.txt') as engDoc:\n    with open('../Data/Datasets/Europarl/Documents/ep-11-06-23-004/alignments/ali.sym') as fa:\n        with open('../Data/Datasets/Europarl/Documents/ep-11-06-23-004/text/ep-11-06-23-004-sv.txt') as sweDoc:\n            for lineAli in fa:\n                lineEngDoc = engDoc.readline()\n                lineSweDoc = sweDoc.readline()\n                tokEng = tokenizerEng(lineEngDoc)\n                tokSwe = tokenizerSwe(lineSweDoc)\n\n                alignmentListTemp = lineAli.split()\n                for ali in alignmentListTemp:\n                    aliPair = ali.split('-')\n                    key = str(int(aliPair[0]) + startTokenEng)\n                    if alignments.get(key) is None:\n                        alignments[key] = [int(aliPair[1]) + startTokenSwe]\n                    else:\n                        align

In [10]:
spansLists = []
for alignmentDict, doc in zip(alignmentDicts, docs):
    spansListForPart = []
    for index, cluster in enumerate(doc._.coref_clusters):
        clusterSpansList = []
        for mention in doc._.coref_clusters[index].mentions:
            mentionSpan = []
            aligns = []
            for i in range(mention.start, mention.end):
                key = str(i)
                if alignmentDict.get(key) is not None:
                    for wordIndex in alignmentDict.get(key):
                        aligns.append(wordIndex)
                else:
                    continue
            aligns = sorted(set(aligns))
            maxContinuousSpan = []
            if len(aligns) == 0:
                maxContinuousSpan.append(-1)
            elif len(aligns) == 1:
                maxContinuousSpan.append(aligns[0])
                maxContinuousSpan.append(aligns[0] + 1)
            else:
                start = aligns[0]
                end = aligns[0]
                spansInAligns = []
                current = 0

                for i, tokenIndex in enumerate(aligns):
                    if tokenIndex == start + current:
                        end = tokenIndex
                        current += 1
                        if tokenIndex == aligns[-1]:
                            currentSpan = [start, end + 1]
                            spansInAligns.append(currentSpan)
                        else:
                            continue
                    else:
                        currentSpan = [start, end + 1]
                        spansInAligns.append(currentSpan)
                        start = tokenIndex
                        end = tokenIndex
                        current = 1

                maxlength = 0
                for conSpan in spansInAligns:
                    if conSpan[1] - conSpan[0] > maxlength:
                        maxlength = conSpan[1] - conSpan[0]
                        maxContinuousSpan = conSpan
                    else:
                        continue

            clusterSpansList.append(maxContinuousSpan)
        spansListForPart.append(clusterSpansList)
    spansLists.append(spansListForPart)


In [11]:
import pickle

spansListsTuple = tuple(spansLists)
with open('../Coref/pickleObjects/corefSpansObject', 'wb') as f:
    pickle.dump(spansListsTuple, f)
